# Размещение рекламного объявления созданного в "алиэкспресс рекламная кампания"
это V2. Она находится в корне facebook

In [1]:
# -*- coding: utf-8 -*-

"""! Создание рекламного поста Из алиэкспресс promo """

from pathlib import Path
from selenium.webdriver.common.by import By
from types import SimpleNamespace
from typing import List, Optional
from datetime import datetime, timedelta

import header
from src import gs
from src.utils import get_directory_names, get_filenames
from src.utils import  j_loads, j_loads_ns, j_dumps
from src.utils.printer import pprint
from src.logger.logger import logger
from src.webdriver.driver import Driver, Chrome, Firefox, Edge
from src.suppliers.suppliers_list.aliexpress import AliCampaignEditor

from src.advertisement.facebook import Facebook
from src.advertisement.facebook.scenarios import post_message, upload_media, promote_post

from src.suppliers.suppliers_list.aliexpress.campaign.ali_campaign_editor_jupyter_widgets import JupyterCampaignEditorWidgets

f = Facebook(Driver(Chrome))
d = f.driver

C:\Users\user\Documents\repos\hypotez\src\utils\convertor\xml2dict.py:71: SyntaxWarning: invalid escape sequence '\{'
  result = re.compile("\{(.*)\}(.*)").search(tag)


Enter KeePass master password:  ········


C:\Users\user\Documents\repos\hypotez\src\webdriver\executor.py:401: SyntaxWarning: invalid escape sequence '\{'
  pprint(f"\{ex=}\n{max_attempts=}\n{ex=}")
C:\Users\user\Documents\repos\hypotez\src\suppliers\aliexpress\__init__.py:1: SyntaxWarning: invalid escape sequence '\_'
  """! supplier `aliexpress`
C:\Users\user\Documents\repos\hypotez\src\suppliers\aliexpress\aliapi.py:1: SyntaxWarning: invalid escape sequence '\e'
  """! AliExpress API Scenario


<font color='red'>Осторжно! Дождись полной загрузки, потом переведи фокус на след поле, иначе перезапустится Хром </font> 

In [ ]:
## \file facebook_groups_widgets.py
# -*- coding: utf-8 -*-

"""
Создание Jupyter виджетов для Facebook групп с дропдауном, содержащим URL-адреса групп.

Импортируемые модули:
- display из IPython.display
- j_loads из src.utils.jjson
- widgets из ipywidgets
- Driver из src.webdriver (если требуется для инициализации веб-драйвера)
- gs из src.settings (для пути к файлу JSON)

Функции:
- create_dropdown: Создает дропдаун для выбора групп с отображением URL-адреса и переключением драйвера на выбранный URL.
- get_selected_value: Возвращает текущее выбранное значение из дропдауна.
"""

from IPython.display import display
import ipywidgets as widgets


def create_dropdown(groups_ns):
    """
    Создает дропдаун с URL-адресами групп из JSON и отображает выбранный URL.

    Загрузка данных:
    - Файл JSON загружается с помощью функции j_loads.

    Пример использования:
    >>> create_dropdown(groups)
    """

    # Сбор опций для дропдауна
    options = [
        (f"{getattr(group_info,'group_description', 'No description')} ({group_url})", group_url)
        for group_url, group_info in vars(groups_ns).items()
    ]

    # Создание дропдауна
    dropdown = widgets.Dropdown(
        options=options,
        description='Select Group:',
        style={'description_width': 'initial'}
    )

    # Функция для обработки изменения значения
    def on_value_change(change):
        if change['name'] == 'value':
            selected_url = change['new']
            if selected_url:
                # Переключение драйвера на выбранный URL
                d.get_url(selected_url)
                

    # Наблюдение за изменением значения в дропдауне
    dropdown.observe(on_value_change, names='value')

    # Отображение дропдауна
    display(dropdown)

    return dropdown

def get_selected_value(dropdown):
    """
    Возвращает текущее выбранное значение из дропдауна.

    Args:
        dropdown (widgets.Dropdown): Виджет дропдауна.

    Returns:
        str: Текущее выбранное значение.
    """
    return dropdown.value

# Пример пути к файлу JSON
path_to_grous_file: Path = gs.path.google_drive / 'facebook' / 'groups' / 'my_managed_groups.json'

# Загрузка данных из JSON файла
groups = j_loads(path_to_grous_file)
groups_ns: SimpleNamespace = j_loads_ns(path_to_grous_file)

# Вызов функции для создания дропдауна
dropdown = create_dropdown(groups_ns)


In [ ]:
jupyter_editor = JupyterCampaignEditorWidgets()
jupyter_editor.display_widgets()

In [ ]:
def get_promoted_categories(group_url: str):
    """
    Reads the 'promoted_categories' list for a given group URL.

    Args:
        group_url (str): The URL of the group to read from.

    Returns:
        list: A list of SimpleNamespace objects representing the promotions.
    """
    groups_ns: SimpleNamespace = j_loads_ns(path_to_grous_file)
    if hasattr(groups_ns, group_url):
        group = getattr(groups_ns, group_url)
        #pprint(group)
        if 'promoted_categories' in group.__dict__:
            return group.promoted_categories
        else:
            logger.error("The 'promoted_categories' attribute does not exist.")
    else:
        logger.error("The specified group URL does not exist in the namespace.")

def extract_keys_from_namespace_list(namespace_list):
    """
    Extracts keys from a list of SimpleNamespace objects.

    Args:
        namespace_list (list): A list of SimpleNamespace objects.

    Returns:
        list: A list of keys extracted from the SimpleNamespace objects.
    """
    keys = []
    for obj in namespace_list:
        keys.extend(vars(obj).keys())
    return keys

In [ ]:
## \file ../src/advertisement/facebook/scenarios/post_message.py
# -*- coding: utf-8 -*-

"""! Публикация сообщения из алиэкспресс промо """

from email import message
from socket import timeout
import time
from pathlib import Path
from types import SimpleNamespace
from typing import Dict, List
from selenium.webdriver.remote.webelement import WebElement
from src import gs
from src.webdriver.driver import Driver
from src.utils.jjson import j_loads_ns
from src.logger.logger import logger

# Load locators from JSON file.
locator: SimpleNamespace = j_loads_ns(
    Path(gs.path.src, 'advertisement', 'facebook', 'locators', 'post_message.json')
)

def post_title(d: Driver, category: SimpleNamespace) -> bool:
    """! Sends the title and description of a campaign to the post message box.

    Args:
        d (Driver): The driver instance used for interacting with the webpage.
        category (SimpleNamespace): The category containing the title and description to be sent.

    Returns:
        bool: `True` if the title and description were sent successfully, otherwise `None`.

    Examples:
        >>> driver = Driver(...)
        >>> category = SimpleNamespace(title="Campaign Title", description="Campaign Description")
        >>> post_title(driver, category)
        True
    """
    # Scroll backward in the page
    if not d.scroll(1, 1200, 'backward'):
        logger.error("Scroll failed during post title", exc_info=False)
        return

    # Open the 'add post' box
    if not d.execute_locator(locator = locator.open_add_post_box, timeout = 90):
        logger.error("Failed to open 'add post' box", exc_info=False)
        return

    # Construct the message with title and description
    message = f"{category.title}; {category.description};"

    # Add the message to the post box
    if not d.execute_locator(locator.add_message, message):
        logger.error(f"Failed to add message to post box: {message=}", exc_info=False)
        return

    return True

def upload_media(d: Driver, products: List[SimpleNamespace], no_video: bool = False) -> bool:
    """! Uploads media files to the images section and updates captions.

    Args:
        d (Driver): The driver instance used for interacting with the webpage.
        products (List[SimpleNamespace]): List of products containing media file paths.

    Returns:
        bool: `True` if media files were uploaded successfully, otherwise `None`.

    Raises:
        Exception: If there is an error during media upload or caption update.

    Examples:
        >>> driver = Driver(...)
        >>> products = [SimpleNamespace(local_image_path='path/to/image.jpg', ...)]
        >>> upload_media(driver, products)
        True
    """
    # Step 1: Open the 'add media' form. It may already be open.
    if not d.execute_locator(locator.open_add_foto_video_form): 
        return
    d.wait(0.5)

    # Step 2: Ensure products is a list.
    products = products if isinstance(products, list) else [products]
    ret: bool = True

    # Iterate over products and upload media.
    for product in products:
        media_path = product.local_video_path if hasattr(product, 'local_video_path') and not no_video else product.local_image_path
        try:
            # Upload the media file.
            if d.execute_locator(locator = locator.foto_video_input, message = media_path , timeout = 20):
                d.wait(1.5)
            else:
                logger.error(f"Ошибка загрузки изображения {media_path=}")
                return
        except Exception as ex:
            logger.error("Error in media upload", ex, exc_info=True)
            return

    # Step 3: Update captions for the uploaded media.
    if not d.execute_locator(locator.edit_uloaded_media_button):
        logger.error(f"Ошибка загрузки изображения {media_path=}")
        return
    uploaded_media_frame = d.execute_locator(locator.uploaded_media_frame)
    uploaded_media_frame = uploaded_media_frame[0] if isinstance(uploaded_media_frame, list) else uploaded_media_frame
    d.wait(0.3)

    textarea_list = d.execute_locator(locator.edit_image_properties_textarea)
    if not textarea_list:
        logger.error("Не нашлись поля ввода подписи к изображениям")
        return
    # Update image captions.
    if not update_images_captions(d, products, textarea_list):
        return

    return ret


def update_images_captions(d: Driver, products: List[SimpleNamespace], textarea_list: List[WebElement]) -> bool:
    """! Adds descriptions to uploaded media files.

    Args:
        d (Driver): The driver instance used for interacting with the webpage.
        products (List[SimpleNamespace]): List of products with details to update.
        textarea_list (List[WebElement]): List of textareas where captions are added.

    Raises:
        Exception: If there's an error updating the media captions.
    """
    local_units = j_loads_ns(Path(gs.path.src / 'advertisement' / 'facebook' / 'scenarios' / 'translations.json'))

    def handle_product(product: SimpleNamespace, textarea_list: List[WebElement], i: int) -> bool:
        """! Handles the update of media captions for a single product.

        Args:
            product (SimpleNamespace): The product to update.
            textarea_list (List[WebElement]): List of textareas where captions are added.
            i (int): Index of the product in the list.
        """
        direction = getattr(local_units.LOCALE, product.language, "LTR")
        message = ""

        # Add product details to message.
        try:
            if direction == "LTR":
                if hasattr(product, 'product_title'):
                    message += f"{product.product_title}\n"
                if hasattr(product, 'original_price'):
                    message += f"{getattr(local_units.original_price, product.language)}: {product.original_price}\n"
                if hasattr(product, 'sale_price'):
                    message += f"{getattr(local_units.sale_price, product.language)}: {product.sale_price}\n"
                if hasattr(product, 'discount'):
                    message += f"{getattr(local_units.discount, product.language)}: {product.discount}\n"
                if hasattr(product, 'evaluate_rate'):
                    message += f"{getattr(local_units.evaluate_rate, product.language)}: {product.evaluate_rate}\n"
                if hasattr(product, 'promotion_link'):
                    message += f"{getattr(local_units.promotion_link, product.language)}: {product.promotion_link}\n"
                if hasattr(product, 'tags'):
                    message += f"{getattr(local_units.tags, product.language)}: {product.tags}\n"
                message += f"{getattr(local_units.COPYRIGHT, product.language)}"
                
            else:  # RTL direction
                if hasattr(product, 'product_title'):
                    message += f"\n{product.product_title}"
                if hasattr(product, 'original_price'):
                    message += f"\n{product.original_price} :{getattr(local_units.original_price, product.language)}"
                if hasattr(product, 'discount'):
                    message += f"\n{product.discount} :{getattr(local_units.discount, product.language)}"
                if hasattr(product, 'sale_price'):
                    message += f"\n{product.sale_price} :{getattr(local_units.sale_price, product.language)}"
                if hasattr(product, 'evaluate_rate'):
                    message += f"\n{product.evaluate_rate} :{getattr(local_units.evaluate_rate, product.language)}"
                if hasattr(product, 'promotion_link'):
                    message += f"\n{product.promotion_link} :{getattr(local_units.promotion_link, product.language)}"
                if hasattr(product, 'tags'):
                    message += f"\n{product.tags} :{getattr(local_units.tags, product.language)}"
                message += f"\n{getattr(local_units.COPYRIGHT, product.language)}"
                
        except Exception as ex:
            logger.error("Error in message generation", ex, exc_info=True)

        # Send message to textarea.
        try:
           textarea_list[i].send_keys(message) 
           return True
        except Exception as ex:
            logger.error("Error in sending keys to textarea", ex)

    # Process products and update their captions.
    for i, product in enumerate(products):
        if not handle_product(product, textarea_list, i):
            logger.debug(f"Неудача ввода в поле #{i}")

    return True


def promote_post(d: Driver, category: SimpleNamespace, products: List[SimpleNamespace], no_video: bool = False) -> bool:
    """! Manages the process of promoting a post with a title, description, and media files.

    Args:
        d (Driver): The driver instance used for interacting with the webpage.
        category (SimpleNamespace): The category details used for the post title and description.
        products (List[SimpleNamespace]): List of products containing media and details to be posted.

    Examples:
        >>> driver = Driver(...)
        >>> category = SimpleNamespace(title="Campaign Title", description="Campaign Description")
        >>> products = [SimpleNamespace(local_image_path='path/to/image.jpg', ...)]
        >>> promote_post(driver, category, products)
    """
    if not post_title(d, category): 
        return
    d.wait(0.5)

    if not upload_media(d, products, no_video): 
        return
    if not d.execute_locator(locator = locator.finish_editing_button): 
        return
    if not d.execute_locator(locator.publish, timeout = 20):
        print("Publishing...")
        return
    return True


In [2]:
from src.advertisement.facebook import FacebookCampaignPromoter
filenames = [
            "ru_usd.json",
            "usa.json",
            "ger_en_eur.json",
            "he_il.json",
            "ru_il.json",
             ]
excluded_filenames = ["my_managed_groups.json",]

promoter = FacebookCampaignPromoter(d, group_file_paths=filenames, no_video=True)

try:
    promoter.run_campaigns()
except KeyboardInterrupt:
    logger.info("Campaign promotion interrupted.")

AttributeError: 'FacebookCampaignPromoter' object has no attribute 'stop'

In [ ]:
"""! Таблица ```pandas``` файсбук групп """
import json
from typing import Dict, Any
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
from pathlib import Path
from src.utils.jjson import j_loads_ns
import webbrowser


def extract_campaign_info(data: Dict[str, Dict[str, Any]]) -> pd.DataFrame:
    """
    Extract campaign information from Facebook groups data.

    Args:
        data (Dict[str, Dict[str, Any]]): A dictionary where each key is a group URL and each value is a dictionary containing group details.

    Returns:
        pd.DataFrame: A DataFrame containing group URLs, descriptions, and campaign details.

    Examples:
        >>> extract_campaign_info({
        ...     "https://www.facebook.com/groups/1524109868489466/": {
        ...         "group_description": "My Group",
        ...         "campaigns": [
        ...             {
        ...                 "campaign_name": "big_save_270624",
        ...                 "promouted_date": "28/06/24 11:50"
        ...             }
        ...         ]
        ...     },
        ...     ...
        ... })
        group_url                                      group_description                    campaign_name    campaign_date
        https://www.facebook.com/groups/1524109868489466/   My Group                  big_save_270624       28/06/24 11:50
    """
    rows = []
    for group_url, group_info in data.items():
        group_description = group_info.get('group_description', 'No description')
        campaigns = group_info.get('campaigns', [])
        
        if not campaigns:
            rows.append({
                'group_url': group_url,
                'group_description': group_description,
                'campaign_name': 'No campaigns',
                'campaign_date': 'N/A'
            })
        else:
            for campaign in campaigns:
                rows.append({
                    'group_url': group_url,
                    'group_description': group_description,
                    'campaign_name': campaign.get('campaign_name', 'N/A'),
                    'campaign_date': campaign.get('promouted_date', 'N/A'),
                })
    
    return pd.DataFrame(rows)

def open_url(change):
    """Open the selected URL in a web browser."""
    url = change['new']
    if url:
       d.get_utl(url)


# Load data and extract campaign info
facebook_groups_data = j_loads(path_to_file)
campaign_info_df = extract_campaign_info(facebook_groups_data)

# Display the DataFrame
display(campaign_info_df)



In [ ]:
#await upload_media(d=d, products=jupyter_editor.products)

In [ ]:
"""! Редактор групп """

# -*- coding: utf-8 -*-
## \file facebook_groups_widgets.py

from ipywidgets import widgets
from IPython.display import display, clear_output
import json
from src.utils.jjson import j_loads_ns

path_to_file: Path = gs.path.google_drive / 'facebook' / 'groups' / 'my_managed_groups.json'
data = j_loads(path_to_file)

def update_data(url: str, new_url: str, description: str, area: str, campaigns: str) -> None:
    """Update the data dictionary with new values.

    Args:
        url (str): The original URL of the Facebook group.
        new_url (str): The new URL of the Facebook group.
        description (str): The new group description.
        area (str): The new area.
        campaigns (str): The new campaigns list in JSON format.
    """
    if url in data:
        # Remove old URL and add new URL with updated data
        data[new_url] = data.pop(url)
        data[new_url]['group_description'] = description
        data[new_url]['area'] = area
        data[new_url]['campaigns'] = json.loads(campaigns)
        print(f"Updated data for {url} to {new_url}")
    else:
        print(f"URL {url} not found in data.")

def create_widgets_for_url(url: str) -> widgets.Output:
    """Create an Output widget containing widgets for editing the data of a specific URL.

    Args:
        url (str): The URL of the Facebook group to edit.

    Returns:
        widgets.Output: An Output widget containing all the widgets for the given URL.
    """
    output = widgets.Output()

    with output:
        if url not in data:
            print(f"URL {url} not found in data.")
            return

        group = data[url]

        # Create widgets
        url_widget = widgets.Text(
            value=url,
            description='URL:',
            layout=widgets.Layout(width='500px')
        )
        description_widget = widgets.Text(
            value=group.get('group_description', ''),
            description='Description:',
            layout=widgets.Layout(width='500px')
        )
        area_widget = widgets.Text(
            value=group.get('area', ''),
            description='Area:',
            layout=widgets.Layout(width='500px')
        )
        campaigns_widget = widgets.Text(
            value=json.dumps(group.get('campaigns', []), indent=2),
            description='Campaigns:',
            layout=widgets.Layout(width='500px')
        )

        # Update button
        update_button = widgets.Button(description="Update Data")

        def on_update_button_clicked(b: widgets.Button) -> None:
            new_url = url_widget.value
            update_data(url, new_url, description_widget.value, area_widget.value, campaigns_widget.value)
            display_all_widgets()

        update_button.on_click(on_update_button_clicked)

        # Display widgets
        display(url_widget, description_widget, area_widget, campaigns_widget, update_button)

    return output

def display_all_widgets() -> None:
    """Display widgets for all URLs."""
    clear_output(wait=True)
    
    # Create and display widgets for each URL
    all_widgets = [create_widgets_for_url(url) for url in data.keys()]
    for widget in all_widgets:
        display(widget)

# Display all widgets
display_all_widgets()


## Send campaign to my group